## Logistic regression

hi

In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk

from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn import metrics

In [30]:
# LOAD EM UP
df = pd.read_json('data/Sports_and_Outdoors_Reviews_training.json', lines=True)

In [31]:
def text_process(text):
    text = ' '.join(str(text).split())
    return text

In [32]:
# this one takes a lil while so be careful
# group by product and aggregate all the review text
grouped_df = df.groupby("asin")
grouped_lists = grouped_df["summary"].apply(text_process).reset_index()
grouped_lists

,asin,summary
0,00018C9635D55E22BF157AA13E91226F,2045677 Five Stars 2045678 I recommend that yo...
1,0001DE3A462B5C5D33AF3BC1053FC20C,"2142792 Very portable, fairly durable. 2142793..."
2,00022ACC61318C98DA944B9BABD9E5AB,"434812 Great product, poor shipping. 434813 Fi..."
3,0002C8404EBEDA230E4B66A85CEC5503,417817 Small and Cheap 417818 One Star 417819 ...
4,00034EBDF69991833D05B51EE7B11234,91838 Pretty Good 91839 Escrima stick 91840 No...
...,...,...
83744,FFFCB2EBE4D59DA2E256396B12F59FB1,"1262012 For The Price, Resin Pads All the way...."
83745,FFFEE00C6052E1A688F4639D650AA50D,593487 Good purchase 593488 Great balls to go ...
83746,FFFEE7703FE466554E6B5F9C21F09297,593764 Cheapo zipper not for me 593765 Nice ba...
83747,FFFF67EAA043C2DB092DBC8934077556,1496788 No specs -- no deal 1496789 Good tent ...


In [33]:
# get the mean of all the ratings for a product
mean_df = grouped_df['overall'].mean()
mean_df = mean_df.reset_index()
mean_df

,asin,overall
0,00018C9635D55E22BF157AA13E91226F,4.090909
1,0001DE3A462B5C5D33AF3BC1053FC20C,3.909091
2,00022ACC61318C98DA944B9BABD9E5AB,4.698413
3,0002C8404EBEDA230E4B66A85CEC5503,3.400000
4,00034EBDF69991833D05B51EE7B11234,4.214286
...,...,...
83744,FFFCB2EBE4D59DA2E256396B12F59FB1,4.941176
83745,FFFEE00C6052E1A688F4639D650AA50D,4.452381
83746,FFFEE7703FE466554E6B5F9C21F09297,4.310345
83747,FFFF67EAA043C2DB092DBC8934077556,4.600000


In [34]:
# merge product mean ratings + aggregated review text
final_df = pd.merge(grouped_lists, mean_df, on="asin")
final_df

,asin,summary,overall
0,00018C9635D55E22BF157AA13E91226F,2045677 Five Stars 2045678 I recommend that yo...,4.090909
1,0001DE3A462B5C5D33AF3BC1053FC20C,"2142792 Very portable, fairly durable. 2142793...",3.909091
2,00022ACC61318C98DA944B9BABD9E5AB,"434812 Great product, poor shipping. 434813 Fi...",4.698413
3,0002C8404EBEDA230E4B66A85CEC5503,417817 Small and Cheap 417818 One Star 417819 ...,3.400000
4,00034EBDF69991833D05B51EE7B11234,91838 Pretty Good 91839 Escrima stick 91840 No...,4.214286
...,...,...,...
83744,FFFCB2EBE4D59DA2E256396B12F59FB1,"1262012 For The Price, Resin Pads All the way....",4.941176
83745,FFFEE00C6052E1A688F4639D650AA50D,593487 Good purchase 593488 Great balls to go ...,4.452381
83746,FFFEE7703FE466554E6B5F9C21F09297,593764 Cheapo zipper not for me 593765 Nice ba...,4.310345
83747,FFFF67EAA043C2DB092DBC8934077556,1496788 No specs -- no deal 1496789 Good tent ...,4.600000


In [35]:
# function to use for lambda to categorize as awesome, not awesome
def df_iter(overall):
    if overall > 4.5:
        result = 'awesome'
    else:
        result = 'not'
        
    return result

In [36]:
final_df['class'] = final_df.apply(lambda row: df_iter(row['overall']), axis=1)
final_df

,asin,summary,overall,class
0,00018C9635D55E22BF157AA13E91226F,2045677 Five Stars 2045678 I recommend that yo...,4.090909,not
1,0001DE3A462B5C5D33AF3BC1053FC20C,"2142792 Very portable, fairly durable. 2142793...",3.909091,not
2,00022ACC61318C98DA944B9BABD9E5AB,"434812 Great product, poor shipping. 434813 Fi...",4.698413,awesome
3,0002C8404EBEDA230E4B66A85CEC5503,417817 Small and Cheap 417818 One Star 417819 ...,3.400000,not
4,00034EBDF69991833D05B51EE7B11234,91838 Pretty Good 91839 Escrima stick 91840 No...,4.214286,not
...,...,...,...,...
83744,FFFCB2EBE4D59DA2E256396B12F59FB1,"1262012 For The Price, Resin Pads All the way....",4.941176,awesome
83745,FFFEE00C6052E1A688F4639D650AA50D,593487 Good purchase 593488 Great balls to go ...,4.452381,not
83746,FFFEE7703FE466554E6B5F9C21F09297,593764 Cheapo zipper not for me 593765 Nice ba...,4.310345,not
83747,FFFF67EAA043C2DB092DBC8934077556,1496788 No specs -- no deal 1496789 Good tent ...,4.600000,awesome


In [37]:
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(stop_words='english', ngram_range = (1,3), tokenizer=token.tokenize)
text_counts = cv.fit_transform(final_df['summary'])

In [50]:
# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(text_counts, final_df['class'], test_size = 0.25, random_state = 5)

In [51]:
# Support Vector
# SVC_classifier = LinearSVC()
# SVC_classifier.fit(X_train, y_train)
# y_score = SVC_classifier.decision_function(X_test)
# y_pred_SVC = SVC_classifier.predict(X_test)

In [52]:
# checking accuracy
# f1_score_SVC = metrics.f1_score(y_test, y_pred_SVC, average='weighted')
# precision_score_SVC = metrics.average_precision_score(y_test, y_score, average='weighted', pos_label="not")

# print('SVC F1: ' + str('{:04.2f}'.format(f1_score_SVC*100)) + '%')
# print('SVC Precision-Recall: ' + str('{:04.2f}'.format(precision_score_SVC*100)) + '%')

In [57]:
from sklearn.linear_model import LogisticRegression

# X_train, X_test, y_train, y_test = train_test_split(text_counts, final_df['class'], test_size=0.3, random_state=0)
logreg = LogisticRegression(penalty='l2', solver='sag', max_iter=10000)
logreg.fit(X_train, y_train)

# Predicting the results, calculating accuracy
y_pred = logreg.predict(X_test)
print("Accuracy of logistic regression classifier on test set: {:.2f}".format(logreg.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.75


In [56]:
# Compute F-1, precision, recall
f1_score = metrics.f1_score(y_test, y_pred, average='weighted')
print('Logistic regression F1: ' + str('{:04.2f}'.format(f1_score*100)) + '%')
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

Logistic regression F1: 74.83%
              precision    recall  f1-score   support

     awesome       0.71      0.73      0.72      9304
         not       0.78      0.76      0.77     11634

    accuracy                           0.75     20938
   macro avg       0.75      0.75      0.75     20938
weighted avg       0.75      0.75      0.75     20938

